In [2]:
# import libraries
import os
import numpy as np
from tqdm import tqdm

from ifum_stitch import Stitch
from ifum_maskopt import Mask
from ifum_rectify import Rectify
from ifum_calibrate import Calibrate
from ifum_stack import Stack

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# directory containing unprocessed files

# directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240212\\"
directory = "C:\\Users\\daniel\\OneDrive - The University of Chicago\\Documents\\cool lamps\\summer_24\\IFUM data\\ut20240210\\"


# all files included in a single stack, repeat where necessary
# only include string in file that includes all files from single exposure

# data_filenames = ["1156","1157","1158","1162","1163","1164","1168","1169","1170"]
# arc_filenames = ["1161","1161","1161","1167","1167","1167","1172","1172","1172"]
# flat_filenames = ["1160","1160","1160","1166","1166","1166","1171","1171","1171"]
# data_filenames = ["0721","0722","0723","0727","0728","0729","0736","0737","0738"]
# arc_filenames = ["0725","0725","0725","0733","0733","0733","0740","0740","0740"]
# flat_filenames = ["0724","0724","0724","0734","0734","0734","0739","0739","0739"]
# data_filenames = ["1177","1178"]
# arc_filenames = ["1180","1180"]
# flat_filenames = ["1179","1179"]
# data_filenames = ["0746","0747","0748"]
# arc_filenames = ["0745","0745","0745"]
# flat_filenames = ["0744","0744","0744"]
data_filenames = ["0756","0757","0758"]
arc_filenames = ["0760","0760","0760"]
flat_filenames = ["0759","0759","0759"]
# data_filenames = ["1148","1149","1150","1151","1152"]
# arc_filenames = ["1161","1161","1161","1161","1161"]
# flat_filenames = ["1160","1160","1160","1160","1160"]

# mask???
# mode LR,STD,HR
mode = "HR"

# far red, blue, ...
wavelength = "far red"

# bad masks (on scale 1-x)
# bad_blues = [23]
# bad_reds = []
bad_blues = [271,298,351]
bad_reds = [189,216,243]

# stars to use in WCS (list RA,Dec)
# all stars should be present in at least some dithers

# wcs_stars = [[74.8322, -58.6579],
#              [74.8279, -58.6548],
#              [74.8314, -58.6526],
#              [74.8254, -58.6572],
#              [74.8303, -58.6543]]
wcs_stars = [[74.8322, -58.6579],
             [74.8305, -58.6603],
             [74.8308, -58.6587],
             [74.8254, -58.6572],
             [74.8237, -58.6594]]

bin_to_2x1 = True

# things to add...
# plot = False (generic plots)
# debug_mode = False (extra print statements, plots, etc.)
# overwrite = False (if file exists, assume already processed correctly)
# sub_steps = False (show smaller TQDM for each mask where relevant)

In [4]:
# assert statements!!!
bad_masks = [np.array(bad_blues)-1,np.array(bad_reds)-1]
wcs_stars = np.array(wcs_stars)
if mode == "STD":
    total_masks = 552
    mask_groups = 12
    hex_dims = (23,24)
elif mode == "HR":
    total_masks = 864
    mask_groups = 16
    hex_dims = (27,32)
else:
    print("invalid mode")

<h1><strong><span style="color:purple">STITCH</h1>

<span style="color:orange"><strong>creates 2 files, one for each detector, for both the data and arc files. also produces a cosmic ray mask for the data.

In [7]:
# out directory where all files are stored
if not os.path.exists("out"):
    os.makedirs("out")

# stitch and create file
for file in tqdm(data_filenames+arc_filenames+flat_filenames):
    file_to_stitch = Stitch(directory,file,None,None,None,None,None)
    file_to_stitch.load_files()
    file_to_stitch.save_file(bin_to_2x1)
print("stitched files saved")

100%|██████████| 9/9 [00:13<00:00,  1.50s/it]

stitched files saved


In [8]:
# use flat and median filter to get rid of internal bias
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_bias = Stitch(directory,None,None,"b",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
    file_for_bias = Stitch(directory,None,None,"r",datafilename,arcfilename,flatfilename)
    file_for_bias.bias_sub()
print("internal bias solved")

100%|██████████| 3/3 [00:14<00:00,  4.82s/it]

internal bias solved


In [9]:
# create cosmic ray masks
for datafilename in tqdm(data_filenames):
    file_for_cmray = Stitch(directory,None,None,"b",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
    file_for_cmray = Stitch(directory,None,None,"r",datafilename,None,None)
    file_for_cmray.cmray_mask(data_filenames)
print("cosmic ray masks created")

100%|██████████| 3/3 [02:10<00:00, 43.37s/it]

cosmic ray masks created


<h1><strong><span style="color:purple">OPTIMIZE MASK</h1>

<span style="color:orange"><strong>use current mask for single gaussian fits to get better mask guess  

In [ ]:
# first guess; complex guess
# IMPLEMENT RANSAC??? (HBDSCAN?)
for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3,median_filter=(2,9))
    file_for_mask.mask_poly(mask_polys0,40)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    mask_polys0 = file_for_mask.first_guess(3)
    file_for_mask.mask_poly(mask_polys0,40)

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# change the degrees until fully confident with all the fits!
center_deg = 5
sigma_deg = 2

for flatfilename in np.unique(flat_filenames):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)

    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.plot_trace_fits(center_deg,sigma_deg)    

In [8]:
# TODO: calculate sig_mult based on mask non-overlap
sig_mult = 1.5

for flatfilename in tqdm(np.unique(flat_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.get_flat_traces(center_deg,sigma_deg)
    file_for_mask.create_mask(sig_mult)

100%|██████████| 1/1 [02:10<00:00, 130.58s/it]


In [4]:
# optimize arc files
expected_peaks = 25#15
sig_mult = 1.5
optimize = True # depends if not enough lines

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks,optimize=optimize)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(arcfilename,sig_mult,expected_peaks=expected_peaks,optimize=optimize)

100%|██████████| 1/1 [17:01<00:00, 1021.98s/it]


In [5]:
# optimize data files
expected_peaks = 10#25
sig_mult = 1.5
optimize = False # rec

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks,optimize=optimize)
    file_for_mask.get_rots(arcfilename,datafilename,optimize=optimize)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.optimize_trace(datafilename,sig_mult,True,expected_peaks=expected_peaks,optimize=optimize)
    file_for_mask.get_rots(arcfilename,datafilename,optimize=optimize)

100%|██████████| 2/2 [05:19<00:00, 159.61s/it]


In [4]:
# create masks for arc!
sig_mult = 1.5

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,arcfilename)

100%|██████████| 1/1 [02:07<00:00, 127.77s/it]


In [5]:
# create masks for data
sig_mult = 1.5

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_mask = Mask("b",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename,copy=arcfilename)
    
    file_for_mask = Mask("r",flatfilename,bad_masks,total_masks,mask_groups)
    file_for_mask.create_mask(sig_mult,datafilename,copy=arcfilename)

100%|██████████| 2/2 [04:35<00:00, 137.79s/it]


<h1><strong><span style="color:purple">RECTIFY</h1>

<span style="color:orange"><strong>rectify + calibrate

In [ ]:
# TEMP
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect._viz_wl()

  0%|          | 0/2 [00:00<?, ?it/s]

activated


 50%|█████     | 1/2 [00:07<00:07,  7.64s/it]

activated


100%|██████████| 2/2 [00:25<00:00, 12.72s/it]


In [ ]:
# fix sparse should only be used when you're confident that lines will appear in each "quadrant" (= 6 splits along x)
# for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
#     file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
#     file_for_rect.optimize_centers("data")
    
#     file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
#     file_for_rect.optimize_centers("data")

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",fix_sparse=True)

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.optimize_centers("arc",fix_sparse=True)

100%|██████████| 1/1 [05:03<00:00, 303.02s/it]


In [ ]:
# for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
#     file_for_rect = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
#     file_for_rect.rectify("data")

#     file_for_rect = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
#     file_for_rect.rectify("data")

for arcfilename, flatfilename in tqdm(zip(np.unique(arc_filenames), np.unique(flat_filenames)),total=len(np.unique(arc_filenames))):
    file_for_rect = Rectify("b","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

    file_for_rect = Rectify("r","NA",arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_rect.rectify("arc")

  0%|          | 0/1 [00:00<?, ?it/s]

GOOD emission line: 74.54527325997013
GOOD emission line: 204.333426931476
GOOD emission line: 245.69887708503092
GOOD emission line: 323.0748842457042
GOOD emission line: 393.42058864897297
GOOD emission line: 433.3759428912239
GOOD emission line: 445.7317604184248
GOOD emission line: 476.49475076733296
BAD emission line: 484.52622339257783
GOOD emission line: 516.3614690384303
GOOD emission line: 756.9446602184532
GOOD emission line: 808.1577447914631
GOOD emission line: 849.2072768775214
GOOD emission line: 964.4203258718495
GOOD emission line: 1057.6747638310503
BAD emission line: 1063.8896039001866
BAD emission line: 1101.0775292905207
BAD emission line: 1111.91363803719
BAD emission line: 1157.800881575322
BAD emission line: 1270.3563119728365
BAD emission line: 1650.9539368619892
BAD emission line: 1734.5904193647277
BAD emission line: 1745.9491303580157
BAD emission line: 1742.7594775468724
BAD emission line: 1751.052505061452
GOOD emission line: 1769.5271955934084
GOOD emissio

100%|██████████| 1/1 [01:01<00:00, 61.48s/it]


In [ ]:
# this function is used ONLY if you wish to use x rectification also for y
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.overwrite_rect()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.overwrite_rect()

100%|██████████| 2/2 [00:00<00:00,  5.36it/s]


In [ ]:
# calibrate (rectified xs to wls)
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.calib(use_sky=False)

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.calib(use_sky=False)

  0%|          | 0/2 [00:00<?, ?it/s]

BAD arc fit: 2 4387.9296
BAD arc fit: 2 4471.4802
BAD arc fit: 4 4471.4802
BAD arc fit: 6 4471.4802
BAD arc fit: 9 4132.56
BAD arc fit: 10 4713.1457
BAD arc fit: 12 4120.8154
BAD arc fit: 12 4713.1457
BAD arc fit: 13 4471.4802
BAD arc fit: 16 4713.1457
BAD arc fit: 18 4713.1457
BAD arc fit: 18 4132.56
BAD arc fit: 19 4471.4802
BAD arc fit: 28 4471.4802
BAD arc fit: 29 4026.1914
BAD arc fit: 30 4132.56
BAD arc fit: 31 4026.1914
BAD arc fit: 33 4026.1914
BAD arc fit: 35 4026.1914
BAD arc fit: 37 4026.1914
BAD arc fit: 38 4120.8154
BAD arc fit: 39 4026.1914
BAD arc fit: 40 4026.1914
BAD arc fit: 49 4120.8154
BAD arc fit: 57 4026.1914
BAD arc fit: 59 4026.1914
BAD arc fit: 66 4120.8154
BAD arc fit: 72 4132.56
BAD arc fit: 73 4026.1914
BAD arc fit: 80 4471.4802
BAD arc fit: 83 4120.8154
BAD arc fit: 89 4026.1914
BAD arc fit: 91 4026.1914
BAD arc fit: 93 4026.1914
BAD arc fit: 97 4026.1914
BAD arc fit: 103 4120.8154
BAD arc fit: 109 4471.4802
BAD arc fit: 110 4471.4802
BAD arc fit: 111 4132.

 50%|█████     | 1/2 [00:48<00:48, 48.05s/it]

BAD arc fit: 4 4026.1914
BAD arc fit: 5 4026.1914
BAD arc fit: 6 4026.1914
BAD arc fit: 6 4471.4802
BAD arc fit: 8 4026.1914
BAD arc fit: 8 4471.4802
BAD arc fit: 10 4026.1914
BAD arc fit: 11 4026.1914
BAD arc fit: 11 4471.4802
BAD arc fit: 12 4026.1914
BAD arc fit: 13 4026.1914
BAD arc fit: 14 4026.1914
BAD arc fit: 15 4026.1914
BAD arc fit: 16 4026.1914
BAD arc fit: 17 4026.1914
BAD arc fit: 18 4026.1914
BAD arc fit: 19 4026.1914
BAD arc fit: 20 4026.1914
BAD arc fit: 20 4713.1457
BAD arc fit: 21 4026.1914
BAD arc fit: 22 4026.1914
BAD arc fit: 22 4387.9296
BAD arc fit: 23 4026.1914
BAD arc fit: 23 4713.1457
BAD arc fit: 24 4026.1914
BAD arc fit: 25 4026.1914
BAD arc fit: 26 4026.1914
BAD arc fit: 27 4026.1914
BAD arc fit: 27 4387.9296
BAD arc fit: 27 4132.56
BAD arc fit: 28 4026.1914
BAD arc fit: 28 4713.1457
BAD arc fit: 30 4026.1914
BAD arc fit: 30 4713.1457
BAD arc fit: 32 4026.1914
BAD arc fit: 32 4713.1457
BAD arc fit: 34 4026.1914
BAD arc fit: 36 4026.1914
BAD arc fit: 36 4713

100%|██████████| 2/2 [01:40<00:00, 50.04s/it]


In [ ]:
# viz
import matplotlib.pyplot as plt
for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Rectify("b",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

    file_for_calib = Rectify("r",datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

    # plt.show()

100%|██████████| 2/2 [02:00<00:00, 60.36s/it] 


<h1><strong><span style="color:purple">CALIBRATE</h1>

<span style="color:orange"><strong>calibrate

In [ ]:
sig_mult = 1.5
bins = np.arange(4000,6100,1)

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.get_spectra(sig_mult,bins,color="b",use_global=True)
    file_for_calib.get_spectra(sig_mult,bins,color="r",use_global=True)

100%|██████████| 2/2 [45:19<00:00, 1359.64s/it]


In [ ]:
# intenisty calibrate all spectra, combine blue and red amplifiers

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib.intensity_corr(deg=0)

  0%|          | 0/2 [00:00<?, ?it/s]

error in intensity fit


100%|██████████| 2/2 [00:36<00:00, 18.44s/it]


In [ ]:
# ONLY VIZ

for datafilename, arcfilename, flatfilename in tqdm(zip(data_filenames, arc_filenames, flat_filenames),total=len(data_filenames)):
    file_for_calib = Calibrate(datafilename,arcfilename,flatfilename,wavelength,bad_masks,total_masks,mask_groups)
    file_for_calib._viz()

100%|██████████| 2/2 [00:37<00:00, 18.73s/it]


<h1><strong><span style="color:purple">STACK</h1>

<span style="color:orange"><strong>stack dithers

In [4]:
# individually create datacubes to be used in dither stack

files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.hex_to_grid()
files_for_stack.spectra_to_datacube()

hexagon grid: 99.000,100.805
pixel grid: 99,101
9999 pixels


overlap percentages: 100%|██████████| 9999/9999 [00:39<00:00, 254.17it/s]


9932 (99.330%) of pixels overlap with hexagon grid
9428 (94.289%) of pixels overlap fully with hexagon grid


graphing pixels: 100%|██████████| 9999/9999 [00:12<00:00, 802.30it/s]


plotting...


graphing pixels: 100%|██████████| 9999/9999 [00:14<00:00, 680.01it/s]


plotting...


100%|██████████| 99/99 [03:15<00:00,  1.98s/it]


In [ ]:
# transform data to WCS using cross-correlation
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.wcs_datacubes()

calculting 2D cross-correlation shift guesses...
select 5 WCS stars in order; see popup window
optimizing WCS star coordinates...
performing WCS transforms...
	0721 used 5 reference stars
	0722 used 5 reference stars
	0723 used 5 reference stars
	0727 used 5 reference stars
	0728 used 5 reference stars
	0729 used 5 reference stars
	0736 used 5 reference stars
	0737 used 5 reference stars
	0738 used 5 reference stars
WCS transformations complete


In [ ]:
# calibrate intensity for all datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.full_intensity_callibration()

In [ ]:
# stack datacubes
files_for_stack = Stack(data_filenames,bad_masks,total_masks,mask_groups,wcs_stars,hex_dims)
files_for_stack.stack_datacubes()

100%|██████████| 99/99 [1:07:33<00:00, 40.94s/it]


final data cubes saved!
